This is a short demo to show how to use gen.py, which contain various utility functions for linear recurrent sequences.

In [31]:
from gen import *
import math

Let $S$ be the generating series for a linearly recurrent sequence $(s_i)_{i\ge0}$ of order $d$ that is
generated by $s_{i+d} = \sum_j^{d-1} c_j s_{i+j}$. The closed form for such a series is
$s_n = \sum_i r_i^n a_i$ for some $a_i$'s determined by the initial condition. Here, we will start with a simple
example, which is an approximation of the well-known Fibonacci sequence $(0,1,1,2,3,5,...)$. This has the closed for 
$s_i = \frac{\phi^i - (\frac{-1}{\phi})^i}{\sqrt{5}}, \phi \approx 1.618$

In [50]:
D = 2

phi = (1+math.sqrt(5))/2
den = math.sqrt(5)
roots = [phi, -1/phi]
coeffs = [1,-1]

print("Dth:", gen_Dth(D,roots,coeffs)/den)
print("Dth slice:", gen_slice(D,roots,coeffs)*(1/den))

Dth: 1.0
Dth slice: [1. 2.]


If we want to train a generative model, we will want to create training data of the form 
$[(s_{D}, \cdots, s_{D+d}), (s_{D+1}, \cdots, s_{D+d+1})]$. We start with an easy one; we fix 
the coefficients $a_i$'s and generate different slices of it.

In [54]:
n = 20 # number of training data we want
Ds = [ random.randint(0,20) for i in range(n) ]
T = np.array( [ gen_slice(Ds[i] ,roots, coeffs)*(1/den) for i in range(len(Ds)) ] ) # initial seq
L = np.array( [ gen_slice(Ds[i]+1 ,roots, coeffs)*(1/den) for i in range(len(Ds)) ] ) # shifted sequence
print("Ds[0:5]", Ds[0:5])
print("T[0:5]:", T[0:5])
print("L[0:5]:", L[0:5])

Ds[0:5] [18, 20, 9, 16, 12]
T[0:5]: [[ 2584.  4181.]
 [ 6765. 10946.]
 [   34.    55.]
 [  987.  1597.]
 [  144.   233.]]
L[0:5]: [[ 4181.  6765.]
 [10946. 17711.]
 [   55.    89.]
 [ 1597.  2584.]
 [  233.   377.]]


What we really want to learn though, is the generation rule $s_{i+d} = s_{i} + s_{i+1}$. Incidently, the polynomial
$P = x^2 - x - 1$ is called the minimal polynomial of this sequence. To do this, we first generate random coefficients
$a_i$'s:

In [55]:
coeffs = rand_vec(2)
print("coeffs:", coeffs)

coeffs: [-0.74557146 -0.60258833]


Now, we generate many of this and take random slices:

In [57]:
n = 50
Ds = [ random.randint(0,20) for i in range(n) ]
coeffs_lst = [ rand_vec(2) for i in range(n) ]
T = [ gen_slice(Ds[i], roots, coeffs_lst[i] ) for i in range(n) ]
L = [ gen_slice(Ds[i]+1, roots, coeffs_lst[i]) for i in range(n) ]
print("Ds[0:5]", Ds[0:5])
print("T[0:5]:", T[0:5])
print("L[0:5]:", L[0:5])


Ds[0:5] [15, 11, 7, 19, 12]
T[0:5]: [array([-1053.01030894, -1703.80826275]), array([-180.46736832, -292.00504187]), array([22.35952804, 36.18338035]), array([-10002.47470147, -16184.34382749]), array([-343.59186581, -555.94537846])]
L[0:5]: [array([-1703.80826275, -2756.81857169]), array([-292.00504187, -472.47241019]), array([36.18338035, 58.54290839]), array([-16184.34382749, -26186.81852896]), array([-555.94537846, -899.53724426])]
